## Wandb에서 pt 파일을 받아서, ONNX Export 후에 다시 업로드

In [ ]:
"""
----------------------------------------------------------------------------
Filename: neuboat_upload_download.ipynb
Path: tools/neuboat_upload_download.ipynb
Date: 08.09.2023
Author: youngjae.you
Purpose: Wandb Registry에서 Neuboat detection task 관련 최신 pt, onnx, engine 모델을 다운로드하거나 업로드할 수 있는 스크립트
History:
----------------------------------------------------------------------------
"""

: 

### Wandb 설치

In [1]:
# Install the Weights & Biases library
!pip install wandb -qqq

In [2]:
# Log a dataset version as an artifact
import wandb
import os

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/yjyou/.netrc


True

### Latest 모델 다운로드

In [4]:
run = wandb.init()
artifact = run.use_artifact('avikus-cv/model-registry/neuboat:pt', type='model')
artifact_dir = artifact.download()

wandb: Currently logged in as: youngjae-you (avikus-cv). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact neuboat:pt, 88.81MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:5.3


In [16]:
model_path = './artifacts/neuboat:pt/best.pt'

### Export onnx

In [17]:
command = f"python ../export_yoloV5_for_deepstream.py -w {model_path} -s 544 960 --opset 12 --simplify --dynamic"
!{command}


Starting: ./artifacts/neuboat:pt/best.pt
Opening YOLOv5 model

YOLOv5 🚀 b5bb23d Python-3.8.16 torch-1.12.1 CPU

Fusing layers... 
Model summary: 267 layers, 46124433 parameters, 0 gradients, 107.7 GFLOPs

Creating labels.txt file

Exporting the model to ONNX
Simplifying the ONNX model
Done: best.onnx



In [18]:
import os

current_filename = 'best.onnx'
new_filename = 'neuboat.onnx'

os.rename(current_filename, new_filename)

In [19]:
labels_file = 'labels.txt'
labels = []

with open(labels_file, 'r') as file:
    for line in file:
        label = line.strip()  # Remove leading/trailing whitespaces
        labels.append(label)

print(labels)

['Buoy', 'Motor Boat', 'Channel Marker', 'Speed Warning Sign']


### Upload Wandb Artifact

In [20]:
art = wandb.Artifact(f"nature-{wandb.run.id}", 
                        type="model",
                        metadata={'format': 'onnx',
                                  'model_input_size': '544 960',
                                  'classes': labels})

art.add_file(new_filename)
art.add_file(labels_file)

### Add aliases to keep track of your best checkpoints over time
wandb.log_artifact(art, aliases=["best", "latest"])

<Artifact nature-mt1msmmg>

In [21]:
run.link_artifact(art, 'avikus-cv/model-registry/neuboat', aliases=['best', 'onnx'])

In [22]:
wandb.finish()